In [1]:
# Import libraries
import psycopg2
import getpass
import numpy as np
import os.path

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
port = '5433'
dbname = 'mimic'
schema = 'mimiciii'

In [4]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
con.commit()
print("Connected!")

Password: ··········


Connected!


In [5]:
# Select admission IDs from MIMIC III that correspond to first time admissions, for patient 18 years old or older
query = \
"""
WITH admission_time AS
(
    SELECT a.subject_id, a.hadm_id, a.admittime, p.dob,
        MIN(a.admittime) OVER (PARTITION BY a.subject_id) AS first_admittime,
        ROUND((CAST(a.admittime AS date) - CAST(p.dob AS date))/365.242,2) AS admit_age
    FROM admissions a
    INNER JOIN patients p
    ON a.subject_id = p.subject_id
)
, age AS
(
    SELECT subject_id, hadm_id, admit_age,
        admittime = first_admittime AS first_admit_flag
    FROM admission_time
)
SELECT hadm_id
FROM age
WHERE admit_age >= 18 AND first_admit_flag = true;
"""

# Execute the query and fetch the result
cur.execute(query)
con.commit()
data = cur.fetchall()

# Store the result of the query as a list
admission_ids = []
for row in data:
    admission_ids.append(row[0])

admission_ids_txt = ','.join(map(str, admission_ids))

In [6]:
# Make sure there are no duplicates in admission_ids
if len(admission_ids) == len(set(admission_ids)):
    print("There are {} entries in admission_ids, and there are no duplicates.".format(len(admission_ids)))
else:
    print("There are {} entries in admission_ids, but only {} are unique.".format(len(admission_ids), len(set(admission_ids))))

There are 38549 entries in admission_ids, and there are no duplicates.


In [7]:
# Save to admission_ids.npy
if not os.path.exists('./res'):
    os.makedirs('./res')

tosave = {'admission_ids':admission_ids, 'admission_ids_txt': admission_ids_txt}
np.save('res/admission_ids.npy',tosave)
print("Saved!")

Saved!
